# The knapsack problem example

The purpose of this tutorial is to demonstrate how to formulate and solve a
simple optimization problem.

## Required packages

This tutorial requires the following packages:

In [1]:
import Pkg
using LinearAlgebra
using Random
using Plots

In [2]:
const n = 250
const k =8
file = "dsjc250.1.col.txt"

"dsjc250.1.col.txt"

In [3]:
adjacence_list=Dict{Int, Vector{Int}}()
adjacence_list[1]=[2, 3, 4,5,6]
adjacence_list[2]=[1, 3, 4, 5]
adjacence_list[3]=[1, 2, 4]
adjacence_list[4]=[1, 2, 3]
adjacence_list[5]=[1, 2]
adjacence_list[6]=[1]
adjacence_list

"""
Calcul des degrés et l'ordre décroissant
"""
degree=zeros(Int, n)
for i in 1:n
    degree[i]=size(adjacence_list[i])[1]
end

indice_degree = sortperm(degree, rev=true)

KeyError: KeyError: key 7 not found

In [3]:
function define_adjacence_list(file, n)
    """
    Lecture du fichier et Création liste d'adjacence
    """
    file_read = open(file,"r");
    lines = readlines(file_read)
    adjacence_list = Dict{Int, Vector{Int}}()
    for line in lines
        st=split(line, " ")
        if st[1]=="e"
            u=parse(Int, st[2])
            v=parse(Int, st[3])
            if haskey(adjacence_list, u)
                push!(adjacence_list[u], v)
            else
                adjacence_list[u] = [v]
            end
        
            if haskey(adjacence_list, v)
                push!(adjacence_list[v], u)
            else
                adjacence_list[v] = [u]
            end
        end
    end
    return adjacence_list
end

adjacence_list=define_adjacence_list(file, n)

"""
Calcul des degrés et l'ordre décroissant
"""
degree=zeros(Int, n)
for i in 1:n
    degree[i]=size(adjacence_list[i])[1]
end

indice_degree = sortperm(degree, rev=true)

250-element Vector{Int64}:
 238
   1
 154
  69
 117
 241
   5
  35
  65
  87
   ⋮
 170
 184
 216
 239
  78
 178
 246
 142
 185

In [4]:
function random_coloring(n, k, seed=0)
    """
    Coloriage aléatoire
    """
    Random.seed!(seed)
    return rand(1:k, n)
end

random_coloring (generic function with 2 methods)

In [4]:
"""
Algo glouton qui parcourt les sommets selon la degré
"""

function min_color_glouton(adjacence_list, Color, k, i)
    color_used=zeros(Int, k)
    for j in adjacence_list[i]
        if Color[j]!=0
            color_used[Color[j]]+=1
        end
    end
    for index_color in 1:k
        if color_used[index_color]==0
            return index_color
        end
    end
    return argmin(color_used)
end




function glouton(adjacence_list, indice_degree, n, k)
    Color =zeros(Int, n)
    for i in indice_degree
        Color[i]=min_color_glouton(adjacence_list, Color, k, i)
    end
    return Color
end

glouton (generic function with 1 method)

In [5]:
function count_conflict(adjacence_list, Color, n)
    """
    Compte le nombre de conflits dans une solution
    """
    Count=0
    list_conflit=[] #Liste des sommets en conflit
    bool_list_conflit=[0 for i in 1:n] # 1 si le sommet est en conflit, 0 sinon 
    for i in 1:n
        for j in adjacence_list[i]
            if Color[i]==Color[j] && j>i
                Count+=1
                if bool_list_conflit[i]==0
                    bool_list_conflit[i]=1
                    push!(list_conflit,i)
                end
                if bool_list_conflit[j]==0
                    bool_list_conflit[j]=1
                    push!(list_conflit,j)
                end
            end
        end
    end
    return Count, list_conflit
end

count_conflict (generic function with 1 method)

In [6]:
sol_glouton=glouton(adjacence_list, indice_degree, n, k)
count, list_conflit=count_conflict(adjacence_list, sol_glouton, n)

(44, Any[2, 61, 5, 112, 11, 28, 12, 24, 14, 151  …  190, 176, 238, 181, 237, 184, 185, 188, 233, 241])

In [7]:
function initialize_T_conf(adjacence_list, sol, n, k)
    T_conf=zeros(Int, n, k)
    for i in 1:n
        for j in adjacence_list[i]
            T_conf[j, sol[i]]+=1
        end
    end
    return T_conf
end

function best_voisinbis(adjacence_list, sol, T_conf, indice_degree, n, k)
    index=0
    c_index=0
    c_index_1=0
    max=-n
    for i in indice_degree
        for c in 1:k
            if T_conf[i, sol[i]]-T_conf[i, c]>max && c!=sol[i]
                index=i
                c_index_1=sol[i]
                c_index=c
                max = T_conf[i, sol[i]]-T_conf[i, c]
            end
        end
    end
    for j in adjacence_list[index]
        T_conf[j, c_index_1]-=1
        T_conf[j, c_index]+=1
    end
    return index, c_index, max
end


function amelioration_locale(sol, adjacence_list, indice_degree, degree, n, k)
    best_sol=deepcopy(sol)
    flag=true
    step=0
    T_conf=initialize_T_conf(adjacence_list, best_sol, n, k)
    while flag && step < 200
        index, c_index, diff = best_voisinbis(adjacence_list, best_sol, T_conf, indice_degree, n, k)
        if diff >= 1 
            best_sol[index]=c_index
            # print(diff, ' ', count_conflict(adjacence_list, best_sol, n), '\n')
        else
            flag = false
        end
        step+=1
    end
    return best_sol
end

amelioration_locale (generic function with 1 method)

In [8]:
count_conflict(adjacence_list, amelioration_locale(sol_glouton, adjacence_list, indice_degree, degree, n, k), n)

(35, Any[2, 61, 5, 112, 11, 28, 12, 24, 14, 151  …  227, 176, 238, 181, 237, 184, 185, 188, 233, 241])

In [9]:
function best_voisin_tabou(adjacence_list, sol, T_conf, tabou_matrix, indice_degree, step, n, k, val_act, val_min)
    index=0
    c_index=0
    c_index_1=0
    max=-n
    for i in indice_degree
        for c in 1:k
            if tabou_matrix[i, c]<=step || T_conf[i, sol[i]]-T_conf[i, c]>=(val_act-val_min)/2+1
                if T_conf[i, sol[i]]-T_conf[i, c]>max && c!=sol[i]
                    index=i
                    c_index_1=sol[i]
                    c_index=c
                    max = T_conf[i, sol[i]]-T_conf[i, c]
                end
            end
        end
    end
    if index==0
        print(step, "ERROR")
    end
    for j in adjacence_list[index]
        T_conf[j, c_index_1]-=1
        T_conf[j, c_index]+=1
    end
    return index, c_index, max
end

best_voisin_tabou (generic function with 1 method)

In [31]:
function tabou(adjacence_list, indice_degree, degree, n, k)
    sol=glouton(adjacence_list, indice_degree, n, k)
    best_sol=deepcopy(sol)
    Count_init, list_conflit=count_conflict(adjacence_list, sol,n)
    Count=[Count_init]
    min_count=Count_init
    step=0
    T_conf=initialize_T_conf(adjacence_list, best_sol, n, k)
    length_tabou=1
    tabou_matrix=zeros(Int, n,k)
    cycle=Dict{Tuple, Tuple}()
    for i in 1:n
        for c in 1:k
            cycle[i,c]=(i,c)
        end
    end
    last_change=(1,1)
    last_change_tabou=0
    while step < 5000000
        if Random.rand(1:500)==1
            tabou_matrix=zeros(Int, n,k)
            length_tabou=1
        end
        index, c_index, diff = best_voisin_tabou(adjacence_list, sol, T_conf, tabou_matrix, indice_degree, step, n, k, last(count), min_count)
        sol[index]=c_index
        if cycle[last_change]==(index, c_index)
            if length_tabou <n*k/6 && step > last_change_tabou+length_tabou
                length_tabou+=k
                last_change_tabou=step
            end
        end
        tabou_matrix[index, c_index]=step+length_tabou
        Count_step=last(Count)-diff
        push!(Count, Count_step)
        if Count_step < min_count
            min_count=Count_step
            best_sol=deepcopy(sol)
            length_tabou=1
            tabou_matrix=zeros(Int, n,k)
            last_change_tabou=step
            if min_count==0
                return best_sol, Count
            end
        end
        cycle[last_change]=(index, c_index)
        last_change=(index,c_index)
        step+=1
    end
    return best_sol, Count
end

tabou (generic function with 1 method)

In [32]:
sol_tabou, Count_tabou=tabou(adjacence_list, indice_degree, degree, n, k)
# minimum(Count_tabou)
# plot(Count_tabou)
count_conflict(adjacence_list, sol_tabou, n)

(0, Any[])

In [23]:
# plot(Count_tabou)